### Задание

Попробуйте поработать с датасетом юридических текстов. В датасете всего две важных колонки признаков: заголовок дела и его текст, а целевая переменная - case_outcome (мультиклассовая классификация).

В базовом варианте можно оставить только текст дела, если хотите поинтереснее - можно попробовать распарсить case_title, добыв оттуда дополнительные признаки.

In [5]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


https://www.kaggle.com/datasets/amohankumar/legal-text-classification-dataset

In [6]:
data = pd.read_csv('/content/gdrive/My Drive/data_stud/machlearn/legal_text_classification.csv')
data.head()

,case_id,case_outcome,case_title,case_text
0,Case1,cited,Alpine Hardwood (Aust) Pty Ltd v Hardys Pty Lt...,Ordinarily that discretion will be exercised s...
1,Case2,cited,Black v Lipovac [1998] FCA 699 ; (1998) 217 AL...,The general principles governing the exercise ...
2,Case3,cited,Colgate Palmolive Co v Cussons Pty Ltd (1993) ...,Ordinarily that discretion will be exercised s...
3,Case4,cited,Dais Studio Pty Ltd v Bullett Creative Pty Ltd...,The general principles governing the exercise ...
4,Case5,cited,Dr Martens Australia Pty Ltd v Figgins Holding...,The preceding general principles inform the ex...


In [4]:
print (data['case_outcome'].unique())

['cited' 'applied' 'followed' 'referred to' 'related' 'considered'
 'discussed' 'distinguished' 'affirmed' 'approved']


In [7]:
data.loc[data["case_outcome"] == "cited", "case_outcome"] = 0
data.loc[data["case_outcome"] == "applied", "case_outcome"] = 1
data.loc[data["case_outcome"] == "followed", "case_outcome"] = 2
data.loc[data["case_outcome"] == "referred to", "case_outcome"] = 3
data.loc[data["case_outcome"] == "related", "case_outcome"] = 4
data.loc[data["case_outcome"] == "considered", "case_outcome"] = 5
data.loc[data["case_outcome"] == "discussed", "case_outcome"] = 6
data.loc[data["case_outcome"] == "distinguished", "case_outcome"] = 7
data.loc[data["case_outcome"] == "affirmed", "case_outcome"] = 8
data.loc[data["case_outcome"] == "approved", "case_outcome"] = 9

In [7]:
data

,case_id,case_outcome,case_title,case_text
0,Case1,0,Alpine Hardwood (Aust) Pty Ltd v Hardys Pty Lt...,Ordinarily that discretion will be exercised s...
1,Case2,0,Black v Lipovac [1998] FCA 699 ; (1998) 217 AL...,The general principles governing the exercise ...
2,Case3,0,Colgate Palmolive Co v Cussons Pty Ltd (1993) ...,Ordinarily that discretion will be exercised s...
3,Case4,0,Dais Studio Pty Ltd v Bullett Creative Pty Ltd...,The general principles governing the exercise ...
4,Case5,0,Dr Martens Australia Pty Ltd v Figgins Holding...,The preceding general principles inform the ex...
...,...,...,...,...
24980,Case25203,0,Reches Pty Ltd v Tadiran Pty Ltd (1998) 85 FCR...,That is not confined to persons who control th...
24981,Case25204,0,Sir Lindsay Parkinson &amp; Co Ltd v Triplan L...,Once the threshold prescribed by s 1335 is sat...
24982,Case25205,0,Spiel v Commodity Brokers Australia Pty Ltd (I...,Once the threshold prescribed by s 1335 is sat...
24983,Case25206,7,"Tullock Ltd v Walker (Unreported, Supreme Cour...",Given the extent to which Deumer stands to gai...


In [8]:
data = data.drop(['case_id', 'case_title'], axis=1)

In [9]:
data

,case_outcome,case_text
0,0,Ordinarily that discretion will be exercised s...
1,0,The general principles governing the exercise ...
2,0,Ordinarily that discretion will be exercised s...
3,0,The general principles governing the exercise ...
4,0,The preceding general principles inform the ex...
...,...,...
24980,0,That is not confined to persons who control th...
24981,0,Once the threshold prescribed by s 1335 is sat...
24982,0,Once the threshold prescribed by s 1335 is sat...
24983,7,Given the extent to which Deumer stands to gai...


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [16]:
data = data.dropna()

X = data['case_text'].astype('str')
y = data['case_outcome'].astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [17]:
print (X_train.shape, ' ', y_train.shape, ' ', X_test.shape, ' ', y_test.shape)

(17366,)   (17366,)   (7443,)   (7443,)


In [12]:
from sklearn.utils.class_weight import compute_class_weight

w = compute_class_weight(class_weight="balanced", classes=[0,1,2,3,4,5,6,7,8,9], y=y) #вычисление весов
print (w)
nonbinclass=[0,1,2,3,4,5,6,7,8,9]
res = {nonbinclass[i]: w[i] for i in range(len(nonbinclass))} #делаем словарь...
print (res)

[ 0.20486375  1.01759639  1.10164298  0.56862251 22.15089286  1.46021189
  2.4370334   4.11426202 23.40471698 22.9712963 ]
{0: 0.20486374896779522, 1: 1.0175963904840033, 2: 1.1016429840142097, 3: 0.568622507449003, 4: 22.150892857142857, 5: 1.4602118893466745, 6: 2.4370333988212183, 7: 4.114262023217247, 8: 23.404716981132076, 9: 22.971296296296295}


In [13]:

import nltk
from nltk.corpus import stopwords
from string import punctuation

nltk.download('stopwords')
noise = stopwords.words('english') + list(punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
from sklearn.metrics import accuracy_score, classification_report
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')

vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=noise)
bow = vec.fit_transform(X_train)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", '``', 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


In [20]:
model = LogisticRegression(solver='liblinear', class_weight=res)
model.fit(bow, y_train)
pred = model.predict(vec.transform(X_test))
print(classification_report(pred, y_test))

#результаты не очень, но я пытался... больно долго каждый раз вычисляет

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.70      0.67      0.68      3762
           1       0.25      0.27      0.26       652
           2       0.29      0.33      0.31       606
           3       0.46      0.49      0.47      1243
           4       0.38      0.57      0.45        21
           5       0.30      0.30      0.30       526
           6       0.30      0.27      0.28       337
           7       0.38      0.33      0.35       205
           8       0.57      0.44      0.49        39
           9       0.27      0.21      0.24        52

    accuracy                           0.52      7443
   macro avg       0.39      0.39      0.38      7443
weighted avg       0.53      0.52      0.52      7443

